In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image

from splat.test.create_image_cpu import Camera, Gaussian, create_image

torch.autograd.set_detect_anomaly(True)

point_3d = torch.tensor([[0.09999, 0.1, -0.2]]).requires_grad_(True)
color = torch.tensor([0.8, 0.8, 0.8]).requires_grad_(True)
opacity = torch.tensor([0.5]).requires_grad_(True)
covariance = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]) * .0001
covariance = covariance.view(1, 3, 3)

gaussian = Gaussian(
    mean=point_3d, 
    covariance=covariance, 
    color=color, 
    opacity=opacity
    )

focal_x = torch.tensor([100.0])
focal_y = torch.tensor([100.0])
width = 32
height = 32
camera_rotation = torch.tensor([1, 0, 0, 0]).unsqueeze(0)
camera_translation = torch.tensor([[-0.1, -0.1, 0.0]])

camera = Camera(
    focal_x=focal_x,
    focal_y=focal_y,
    c_x=0.0,
    c_y=0.0,
    width=width,
    height=height,
    camera_rotation=camera_rotation,
    camera_translation=camera_translation,
)
gt_image = Image.open("gt.png")
gt_image = np.array(gt_image) / 255.0

optimizer = torch.optim.Adam([point_3d, color, opacity], lr=0.000005)

for i in range(50):
    gt_image_ = Image.open("gt.png")
    gt_image = torch.tensor(np.array(gt_image_) / 255.0)
    optimizer.zero_grad()
    image, point_ndc = create_image(camera, gaussian, height, width)
    loss = (image - gt_image).pow(2).mean()
    loss.backward()
    # if i == 0:
    #     point_3d.grad[0, 1] *= -1
    optimizer.step()
    # optimizer.zero_grad()
    print("point_3d.grad", point_3d.grad)
    print("color.grad", color.grad)
    print("opacity.grad", opacity.grad)
    print("loss", loss, "\n\n")

point_ndc tensor([[13.0785, 15.5000]], grad_fn=<CatBackward0>)
gaussian.color tensor([0.8000, 0.8000, 0.8000], requires_grad=True)
gaussian.opacity tensor([0.5000], requires_grad=True)
point_3d.grad tensor([[-4.2065e+02,  6.0079e-07,  2.4362e-02]])
color.grad tensor([0.0010, 0.0010, 0.0010])
opacity.grad tensor([0.0009])
loss tensor(0.0022, dtype=torch.float64, grad_fn=<MeanBackward0>) 


point_ndc tensor([[14.2893, 14.3091]], grad_fn=<CatBackward0>)
gaussian.color tensor([0.8000, 0.8000, 0.8000], requires_grad=True)
gaussian.opacity tensor([0.5000], requires_grad=True)
point_3d.grad tensor([[-2.1670e+02, -2.1315e+02,  1.3837e-02]])
color.grad tensor([0.0006, 0.0006, 0.0006])
opacity.grad tensor([0.0005])
loss tensor(0.0011, dtype=torch.float64, grad_fn=<MeanBackward0>) 


point_ndc tensor([[15.4224, 15.2095]], grad_fn=<CatBackward0>)
gaussian.color tensor([0.8000, 0.8000, 0.8000], requires_grad=True)
gaussian.opacity tensor([0.5000], requires_grad=True)
point_3d.grad tensor([[-1.4278e

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image

from splat.test.create_image_cpu import Camera, Gaussian
from splat.test.create_image_custom import create_image

torch.autograd.set_detect_anomaly(True)

point_3d = torch.tensor([[0.09999, 0.1, -0.2]], dtype=torch.float64).requires_grad_(True)
color = torch.tensor([0.8, 0.8, 0.8], dtype=torch.float64).requires_grad_(True)
opacity = torch.tensor([0.5], dtype=torch.float64).requires_grad_(True)
covariance = torch.tensor([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]], dtype=torch.float64) * .0001
covariance = covariance.view(1, 3, 3)

gaussian = Gaussian(
    mean=point_3d, 
    covariance=covariance, 
    color=color, 
    opacity=opacity
    )

focal_x = torch.tensor([100.0], dtype=torch.float64)
focal_y = torch.tensor([100.0], dtype=torch.float64)
width = torch.tensor([32], dtype=torch.int64)
height = torch.tensor([32], dtype=torch.int64)
camera_rotation = torch.tensor([1, 0, 0, 0], dtype=torch.float64).unsqueeze(0)
camera_translation = torch.tensor([[-0.1, -0.1, 0.0]], dtype=torch.float64)

camera = Camera(
    focal_x=focal_x,
    focal_y=focal_y,
    c_x=0.0,
    c_y=0.0,
    width=width,
    height=height,
    camera_rotation=camera_rotation,
    camera_translation=camera_translation,
)
gt_image = Image.open("gt.png")
gt_image = np.array(gt_image) / 255.0

optimizer = torch.optim.Adam([point_3d, gaussian.color, gaussian.opacity], lr=0.000005)

for i in range(50):
    gt_image_ = Image.open("gt.png")
    gt_image = torch.tensor(np.array(gt_image_) / 255.0)
    optimizer.zero_grad()
    image, point_ndc = create_image(camera, gaussian, height, width)
    loss = (image - gt_image).pow(2).mean()
    loss.backward()
    optimizer.step()
    # optimizer.zero_grad()
    print("point_3d.grad", point_3d.grad)
    print("color.grad", color.grad)
    print("opacity.grad", opacity.grad)
    print("loss", loss, "\n\n")


In [ ]:
import torch
from torch.autograd import gradcheck
from torch.autograd.functional import jacobian
from splat.test.partials import gaussianMeanToPixels, pixelCoordToColor
    
focal_x = torch.tensor([100.0])
focal_y = torch.tensor([100.0])
width = torch.tensor([32])
height = torch.tensor([32])
camera_rotation = torch.tensor([1, 0, 0, 0]).unsqueeze(0)
camera_translation = torch.tensor([[-0.1, -0.1, 0.0]])


camera = Camera(
    focal_x=focal_x,
    focal_y=focal_y,
    c_x=0.0,
    c_y=0.0,
    width=width,
    height=height,
    camera_rotation=camera_rotation,
    camera_translation=camera_translation,
)


test_points = torch.tensor([[0.0, 0.1, -0.2]]).requires_grad_(True)
intrinsic_matrix = camera.intrinsic_matrix
extrinsic_matrix = camera.extrinsic_matrix

inputs = (test_points, width, height, intrinsic_matrix, extrinsic_matrix)
test = gradcheck(gaussianMeanToPixels.apply, inputs, eps=1e-4, atol=1e-3)
print("Gradcheck passed:", test)

In [ ]:
pixel_coords = torch.tensor([[5.0, 4.0]])
gaussian_coords = torch.tensor([[0.09, 0.1]]).requires_grad_(True)
color = torch.tensor([0.4, 0.8, 0.2])
inv_covariance_2d = torch.tensor([[0.004, 0.01], [0.01, 0.004]]).view(1, 2, 2)
current_T = torch.tensor([0.5])
opacity = torch.tensor([0.5])

inputs = (pixel_coords, gaussian_coords, color, inv_covariance_2d, current_T, opacity)
test = gradcheck(pixelCoordToColor.apply, inputs, eps=1e-2, atol=1e-4)
print("Gradcheck passed:", test)